# Tunix Reasoning Agent Demo

This notebook demonstrates the capabilities of the Tunix Reasoning Agent, which uses a multi-step reasoning framework to solve problems:

1. **Understand**: Analyze and comprehend the problem
2. **Plan**: Create a solution strategy
3. **Execute**: Implement the solution
4. **Verify**: Validate the results

## Setup

In [ ]:
# Install the package if needed
# !pip install -e ..

import sys
sys.path.insert(0, '../src')

from tunix_reasoning_agent import ReasoningAgent, ProblemDecomposer, ReasoningMetrics, TunixTrainer
from tunix_reasoning_agent.evaluation import MetricsCalculator
from tunix_reasoning_agent.utils import ReasoningVisualizer

print("✓ Imports successful!")

## Example 1: Basic Problem Solving

Let's start with a simple mathematical problem.

In [ ]:
# Initialize the reasoning agent
agent = ReasoningAgent(model_name="gemini-pro", verbose=True)

# Define a problem
problem1 = """
A farmer has 17 sheep. All but 9 die. How many sheep are left?
"""

# Solve the problem
result1 = agent.solve(problem1, decompose=False)

print("\n" + "="*70)
print("SOLUTION:")
print("="*70)
print(result1['solution'])

## Example 2: Complex Problem with Decomposition

For more complex problems, we can enable problem decomposition.

In [ ]:
problem2 = """
A company wants to optimize its supply chain. They have 5 warehouses, 20 distribution centers,
and 100 retail stores. Each warehouse can supply up to 1000 units per day, and each store needs
50 units per day. Design an efficient distribution strategy that minimizes transportation costs
while ensuring all stores receive their required inventory.
"""

# Solve with decomposition
result2 = agent.solve(problem2, decompose=True, decomposition_strategy="sequential")

## Example 3: Examining the Reasoning Trace

Let's visualize the reasoning process in detail.

In [ ]:
# Create visualizer
visualizer = ReasoningVisualizer()

# Visualize the reasoning trace
print(visualizer.visualize_trace(result2['reasoning_trace'], show_metadata=True))

## Example 4: Problem Decomposition

Explore different decomposition strategies.

In [ ]:
decomposer = ProblemDecomposer()

# Try different strategies
strategies = ["sequential", "hierarchical", "parallel"]

for strategy in strategies:
    decomposition = decomposer.decompose(problem2, strategy=strategy, max_depth=2)
    print(decomposer.visualize_decomposition(decomposition))
    print("\n")

## Example 5: Evaluating Reasoning Quality

Use metrics to evaluate the quality of reasoning.

In [ ]:
# Create metrics calculator
calculator = MetricsCalculator()

# Evaluate results
evaluation1 = calculator.evaluate_result(result1)
evaluation2 = calculator.evaluate_result(result2)

print("\nEvaluation for Problem 1:")
print("-" * 70)
for metric, score in evaluation1['metrics'].items():
    print(f"{metric.capitalize():15} → {score:.3f}")

print("\nEvaluation for Problem 2:")
print("-" * 70)
for metric, score in evaluation2['metrics'].items():
    print(f"{metric.capitalize():15} → {score:.3f}")

## Example 6: Multiple Problems - Batch Processing

Process multiple problems and compare results.

In [ ]:
problems = [
    "What is 25% of 200?",
    "If a train travels 120 km in 2 hours, what is its average speed?",
    "A rectangle has a length of 10 cm and width of 5 cm. What is its area and perimeter?",
    "Explain the difference between machine learning and deep learning.",
    "What are the three laws of thermodynamics?"
]

# Solve all problems
results = []
for i, problem in enumerate(problems, 1):
    print(f"\nSolving problem {i}/{len(problems)}...")
    result = agent.solve(problem, decompose=False)
    results.append(result)

print("\n" + "="*70)
print("All problems solved!")
print("="*70)

## Example 7: Batch Evaluation and Statistics

In [ ]:
# Evaluate all results
batch_evaluation = calculator.evaluate_batch(results)

print("\nBatch Evaluation Summary:")
print("="*70)
print(f"Total problems evaluated: {batch_evaluation['total_results']}")
print("\nAggregate Metrics:")
print("-"*70)

for metric, stats in batch_evaluation['aggregate_metrics'].items():
    if isinstance(stats, dict):
        print(f"\n{metric.upper()}:")
        for stat_name, value in stats.items():
            print(f"  {stat_name}: {value:.3f}")
    else:
        print(f"{metric}: {stats:.3f}")

# Get agent statistics
agent_stats = agent.get_statistics()
print("\n\nAgent Performance Statistics:")
print("="*70)
for key, value in agent_stats.items():
    print(f"{key}: {value}")

## Example 8: Visualization and Comparison

In [ ]:
# Create comparison table
print(visualizer.create_comparison_table(results))

# Create summary report
print(visualizer.create_summary_report(
    results,
    evaluations=batch_evaluation['individual_evaluations']
))

## Example 9: Fine-tuning Preparation

Prepare data for fine-tuning the model.

In [ ]:
from tunix_reasoning_agent.fine_tuning import DatasetBuilder

# Extract problems and solutions
training_problems = [r['problem'] for r in results]
training_solutions = [r['solution'] for r in results]
training_traces = [r['reasoning_trace'] for r in results]

# Initialize trainer
trainer = TunixTrainer(output_dir="../models")

# Prepare training data
training_data = trainer.prepare_training_data(
    problems=training_problems,
    solutions=training_solutions,
    reasoning_traces=training_traces,
    output_file="reasoning_training_data.json"
)

print(f"\nPrepared {len(training_data)} training examples")
print(f"Example:")
print("-"*70)
print(f"Problem: {training_data[0]['problem'][:100]}...")
print(f"Solution: {training_data[0]['solution'][:100]}...")

## Example 10: Dataset Building and Augmentation

In [ ]:
# Create dataset builder
dataset_builder = DatasetBuilder(output_dir="../data/processed")

# Create train/validation split
train_data, val_data = dataset_builder.create_from_examples(
    training_data,
    train_ratio=0.8,
    shuffle=True
)

print(f"Training examples: {len(train_data)}")
print(f"Validation examples: {len(val_data)}")

# Get statistics
train_stats = dataset_builder.get_statistics(train_data)
print("\nTraining Data Statistics:")
print("-"*70)
for key, value in train_stats.items():
    print(f"{key}: {value}")

## Example 11: Custom Problem Types

Test the agent on different types of problems.

In [ ]:
problem_types = {
    "Mathematical": "Calculate the compound interest on $1000 at 5% annual rate for 3 years.",
    "Logical": "All roses are flowers. Some flowers fade quickly. Do all roses fade quickly?",
    "Analytical": "Analyze the pros and cons of remote work versus office work.",
    "Creative": "Design a simple algorithm to recommend movies to users based on their viewing history.",
}

for ptype, problem in problem_types.items():
    print(f"\n{'='*70}")
    print(f"Problem Type: {ptype}")
    print(f"{'='*70}")
    
    result = agent.solve(problem, decompose=False)
    print(f"\nSolution:\n{result['solution'][:300]}...")
    print(f"\nVerified: {result['metadata']['verified']}")
    print(f"Confidence: {result['metadata']['confidence']:.2%}")

## Example 12: Export Results

Export results for further analysis.

In [ ]:
# Export results to JSON
visualizer.export_to_json(
    results,
    "../data/processed/reasoning_results.json",
    pretty=True
)

# Export evaluations
visualizer.export_to_json(
    batch_evaluation,
    "../data/processed/evaluation_results.json",
    pretty=True
)

print("✓ Results exported successfully!")
print("  - reasoning_results.json")
print("  - evaluation_results.json")

## Summary

This notebook demonstrated:

1. ✓ Basic problem solving with the Reasoning Agent
2. ✓ Problem decomposition with different strategies
3. ✓ Reasoning trace visualization
4. ✓ Quality evaluation with comprehensive metrics
5. ✓ Batch processing of multiple problems
6. ✓ Fine-tuning data preparation
7. ✓ Dataset building and management
8. ✓ Testing on various problem types
9. ✓ Results export and analysis

The Tunix Reasoning Agent provides a structured approach to problem-solving with transparent reasoning traces and comprehensive evaluation capabilities.